In [16]:
import warnings
warnings.filterwarnings('ignore')
import sys

import keras
import numpy as np
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import class_weight

In [17]:
datagen = ImageDataGenerator(rotation_range=30, 
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True
                            )

if 'win32' in sys.platform:
    data_directory='F:\\Segmentation_Data\\'
    full_training_directory = data_directory + 'Labelled_imgs\\data_for_generator\\training\\'
    full_validation_directory = data_directory +'Labelled_imgs\\data_for_generator\\validation\\'

In [18]:
batch_size = 32
train_generator = datagen.flow_from_directory(full_training_directory, 
                                              target_size=(128, 128), 
                                              color_mode="rgb", 
                                              batch_size=batch_size)
validation_generator = datagen.flow_from_directory(full_validation_directory,
                                                   target_size=(128, 128), 
                                                   color_mode='rgb',
                                                   batch_size=batch_size, 
                                                   shuffle=False)
num_classes = len(train_generator.class_indices)
num_test_samples = 8524

class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_generator.classes), 
            train_generator.classes)

Found 19890 images belonging to 7 classes.
Found 8524 images belonging to 7 classes.


In [19]:
model = Sequential()
model.add(Conv2D(20, (5,5), activation='relu', input_shape=(128,128,3)))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(50, (5,5), activation='relu'))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(100, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(200, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(500))
model.add(Dropout(rate=0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [6]:

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator,
                    verbose=1, 
                    steps_per_epoch=1000, 
                    epochs=100, 
                    validation_data=validation_generator,
                   validation_steps=300,
                   class_weight=class_weights)

W0821 18:32:16.496249  9168 deprecation_wrapper.py:119] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0821 18:32:16.570051  9168 deprecation.py:323] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1000/1000 [==============================] - 176s 176ms/step - loss: 1.1405 - acc: 0.5794 - val_loss: 0.9296 - val_acc: 0.6420
Epoch 2/100
1000/1000 [==============================] - 128s 128ms/step - loss: 0.9320 - acc: 0.6428 - val_loss: 0.8477 - val_acc: 0.6815
Epoch 3/100
1000/1000 [==============================] - 121s 121ms/step - loss: 0.8226 - acc: 0.6838 - val_loss: 0.8615 - val_acc: 0.6492
Epoch 4/100
1000/1000 [==============================] - 125s 125ms/step - loss: 0.7534 - acc: 0.7107 - val_loss: 0.7134 - val_acc: 0.7299
Epoch 5/100
1000/1000 [==============================] - 130s 130ms/step - loss: 0.6992 - acc: 0.7337 - val_loss: 0.6621 - val_acc: 0.7442
Epoch 6/100
1000/1000 [==============================] - 120s 120ms/step - loss: 0.6619 - acc: 0.7495 - val_loss: 0.6190 - val_acc: 0.7653
Epoch 7/100
1000/1000 [==============================] - 120s 120ms/step - loss: 0.6425 - acc: 0.7543 - val_loss: 0.6132 - val_acc: 0.7654
Epoch 8/100
1000/1000 [====

1000/1000 [==============================] - 119s 119ms/step - loss: 0.3801 - acc: 0.8523 - val_loss: 0.3863 - val_acc: 0.8524
Epoch 60/100
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3811 - acc: 0.8537 - val_loss: 0.4022 - val_acc: 0.8571
Epoch 61/100
1000/1000 [==============================] - 118s 118ms/step - loss: 0.3794 - acc: 0.8524 - val_loss: 0.4137 - val_acc: 0.84536s
Epoch 62/100
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3810 - acc: 0.8547 - val_loss: 0.4103 - val_acc: 0.8466
Epoch 63/100
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3737 - acc: 0.8573 - val_loss: 0.4129 - val_acc: 0.8459
Epoch 64/100
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3790 - acc: 0.8566 - val_loss: 0.4052 - val_acc: 0.8499
Epoch 65/100
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3732 - acc: 0.8574 - val_loss: 0.4064 - val_acc: 0.8487
Epoch 66/100
1000/1000 [=======

In [15]:
# https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045

Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[4148   77    4    1    0    1   10]
 [ 126 1328  128    6    0    3    5]
 [  34  174  805  116    5    5    0]
 [  15   12  125  331   48   37    0]
 [   5    2   11   98   67  105    0]
 [   7    1    5   22   23  212   34]
 [  40    7    1    0    0    0  340]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.95      0.98      0.96      4241
  2_colonies       0.83      0.83      0.83      1596
  3_colonies       0.75      0.71      0.73      1139
  4_colonies       0.58      0.58      0.58       568
  5_colonies       0.47      0.23      0.31       288
  6_colonies       0.58      0.70      0.64       304
     outlier       0.87      0.88      0.88       388

    accuracy                           0.85      8524
   macro avg       0.72      0.70      0.70      8524
weighted avg       0.84      0.85      0.84      8524



## Balance Classes

In [20]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator,
                    verbose=1, 
                    steps_per_epoch=1000, 
                    epochs=1, 
                    validation_data=validation_generator,
                   validation_steps=300,
                   class_weight=class_weights)

Epoch 1/1
1000/1000 [==============================] - 122s 122ms/step - loss: 1.1465 - acc: 0.5805 - val_loss: 0.8989 - val_acc: 0.6723


In [21]:
Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[3296  568  327    5    0    2   43]
 [ 318  473  702   49    1   14   39]
 [  96  146  620  127    5  112   33]
 [ 242   33   38   10    0   24  221]
 [ 273   10    1    0    0    0    4]
 [ 286   16    1    0    0    0    1]
 [ 356   28    1    0    0    0    3]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.68      0.78      0.72      4241
  2_colonies       0.37      0.30      0.33      1596
  3_colonies       0.37      0.54      0.44      1139
  4_colonies       0.05      0.02      0.03       568
  5_colonies       0.00      0.00      0.00       288
  6_colonies       0.00      0.00      0.00       304
     outlier       0.01      0.01      0.01       388

    accuracy                           0.52      8524
   macro avg       0.21      0.23      0.22      8524
weighted avg       0.46      0.52      0.48      8524



## Imbalanced Classes

In [22]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator,
                    verbose=1, 
                    steps_per_epoch=1000, 
                    epochs=1, 
                    validation_data=validation_generator,
                   validation_steps=300,
                   class_weight=class_weights)

Epoch 1/1
1000/1000 [==============================] - 120s 120ms/step - loss: 0.9240 - acc: 0.6460 - val_loss: 0.8217 - val_acc: 0.7030


In [23]:
Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[2740 1107  328   27    0   14   25]
 [ 221  446  550   94   13  257   15]
 [ 760   41   21    3    2   45  267]
 [ 552   15    0    0    0    0    1]
 [ 277   10    0    0    0    0    1]
 [ 290   13    0    0    0    0    1]
 [ 378   10    0    0    0    0    0]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.53      0.65      0.58      4241
  2_colonies       0.27      0.28      0.28      1596
  3_colonies       0.02      0.02      0.02      1139
  4_colonies       0.00      0.00      0.00       568
  5_colonies       0.00      0.00      0.00       288
  6_colonies       0.00      0.00      0.00       304
     outlier       0.00      0.00      0.00       388

    accuracy                           0.38      8524
   macro avg       0.12      0.13      0.13      8524
weighted avg       0.32      0.38      0.34      8524

